In [ ]:
# | default_exp create_climate_data

In [ ]:
# | hide
from nbdev.showdoc import *
from fastcore.test import *
from fastcore.utils import *

ModuleNotFoundError: No module named 'sureau_ecos_py'

In [1]:
# | export
import pandas as pd
from pathlib import Path
from typing import List
import os
import pandera as pa
from pandera.typing import DataFrame, Series
import numpy as np

from sureau_ecos_py.create_modeling_options import create_modeling_options

In [ ]:
# | export


# This class is created for validating the input climate dataframe
# If the data don't follow the structure in the class the function will fail
class ClimateDataValidation(pa.SchemaModel):
    """Schema for validating the climate data"""

    DATE: Series[np.datetime64] = pa.Field(description="date with format 1991/12/23")
    Tair_min: Series[float] = pa.Field(
        description="minimum air temperature of the day (degC)"
    )
    Tair_max: Series[float] = pa.Field(
        description=" maximum air temperature of the day (degC)"
    )
    Tair_mean: Series[float] = pa.Field(
        description="mean air temperature of the day (degC)"
    )
    RG_sum: Series[float] = pa.Field(ge=0, description="global radiation (MJ/m2)")
    PPT_sum: Series[float] = pa.Field(ge=0, description="precipitation (mm)")
    RHair_min: Series[float] = pa.Field(
        ge=0, le=100, description="minimum relative humidity of the day (%)"
    )
    RHair_max: Series[float] = pa.Field(
        ge=0,
        le=100,
        description="maximum relative humidity of the day (%)",
        coerce=True,
    )
    RHair_mean: Series[float] = pa.Field(
        ge=0, le=100, description="mean relative humidity of the day (%)"
    )
    WS_mean: Series[float] = pa.Field(
        ge=0, description="mean wind speed of the day (m/s)"
    )


@pa.check_types(lazy=True)
def create_climate_data(
    file_path: Path,  # Path to the input CSV climate file. i.e. path/to/file/climate.csv
    sep: str = ";",  # CSV file separator can be "," or ";"
    modeling_options: List[
        int
    ] = None,  # Dictionary created using the `create_modeling_options` function
    simulation_parameters: List[
        int
    ] = None,  # Dictionary created using the `create_simulation_parameters` function
) -> DataFrame[ClimateDataValidation]:
    """
    Create a climate data.frame to run SureauR.
    Read input climate data select the desired period and put it in the right format to run `run.SurEauR`
    Also check data consistency and input variables according to modeling options
    (see \code{create.modeling.options} and simulation parameters (see \code{create.simulation.parameters)
    """

    # Read file if it exists and climateData not provided, error otherwise ------------------------------------

    if os.path.exists(file_path):
        try:
            climate_data = pd.read_csv(
                file_path, sep=sep, header=0, parse_dates=["DATE"], dayfirst=True
            )

        except pa.errors.SchemaErrors as err:
            print(err)
    else:
        print(f"file: {file_path}, does not exist, check presence or spelling")

    if modeling_options is False:
        # Create function for extracting the day of the year (from 0 to 365)
        def get_day_of_year(date):
            return pd.Period(date, freq="H").day_of_year

        # Map function over each row and create new column
        climate_data["day_of_year"] = pd.DataFrame(
            map(get_day_of_year, climate_data["DATE"])
        )

        # Get the day of the month (from 1 to 31)
        climate_data["day_of_month"] = pd.DatetimeIndex(climate_data["DATE"]).day

        # Get the month (from 1 to 12)
        climate_data["month"] = pd.DatetimeIndex(climate_data["DATE"]).month

        # Get the year
        climate_data["year"] = pd.DatetimeIndex(climate_data["DATE"]).year

    # io <- climate_data$Year >= simulation_parameters$startYearSimulation & climate_data$Year <= simulation_parameters$endYearSimulation

    return climate_data

In [ ]:
create_climate_data(
    "./test_folder/climat_example.csv", sep=";", modeling_options=False
).head()

,DATE,Tair_min,Tair_max,Tair_mean,RG_sum,RHair_min,RHair_max,RHair_mean,WS_mean,PPT_sum,day_of_year,day_of_month,month,year
0,1990-01-01,-4.4,-2.1,-3.25,1.95,96.915145,96.915145,96.915145,0.3,0.0,1,1,1,1990
1,1990-01-02,-3.9,0.0,-1.95,3.22,96.915145,96.915145,96.915145,0.4,0.0,2,2,1,1990
2,1990-01-03,-1.9,0.5,-0.70,1.03,96.915145,96.915145,96.915145,1.5,0.0,3,3,1,1990
3,1990-01-04,-2.0,-0.4,-1.20,1.13,96.915145,96.915145,96.915145,1.2,0.0,4,4,1,1990
4,1990-01-05,-0.7,2.8,1.05,1.14,96.537691,96.537691,96.537691,1.4,0.4,5,5,1,1990
